In [1]:
from cot import Collection

# 1) Dataset load and selecting random sample
collection = Collection(["worldtree"], verbose=False)
collection_worldtree_10 = collection.select(split="train", number_samples=10)

# 2) Language Model generates chains of thought and then extracts answers
config={
    "debug": False,
    "instruction_keys": ['qa-01'],
    "cot_trigger_keys": ['kojima-01'],
    "answer_extraction_keys": ['kojima-A-D']
}
collection_worldtree_10.generate(config=config)

# 3) Performance evaluation
collection_worldtree_10.evaluate()


Loading worldtree...

            You are about to  call an external API  in total 20 times, which  may produce costs .
            Number API calls for CoT generation: n_samples 10 * n_instruction_keys 1 * n_cot_trigger_keys 1
            Number API calls for answer extraction: n_samples 10 * n_instruction_keys 1 * n_cot_trigger_keys 1 * n_answer_extraction_keys 1
            Do you want to continue? y/n
            
{'accuracy': {'qa-01_kojima-01_kojima-A-D': 0.8}}
